In [1]:
import pandas as pd
import random
import os
import numpy as np
import re
import librosa

In [56]:
def get_immediate_files(a_dir):
    return [name for name in os.listdir(a_dir) if os.path.isfile(os.path.join(a_dir, name))]

In [57]:
csv_dir = '../../../../dataset/metadata/meta_sm.csv'  #'/kuacc/users/bbiner21/input.csv'
dataset_dir = '/datasets/xeno_canto/sm_dataset/'#'/kuacc/users/bbiner21/ast/egs/custom/data'
meta_dir = './data'

In [58]:
data = pd.read_csv(csv_dir) 
print("Initial data shape: ")
print(data.shape)
data2 = pd.read_csv(csv_dir)

Initial data shape: 
(9775, 11)


In [59]:
data = data.drop(["type", "latitude","longitude","scientific_name","common_name","url"], axis=1)


In [60]:
data

,primary_label,secondary_label,time,filename,q
0,lesred1,[],08:00,693048.wav,A
1,lesred1,[],11:00,692820.wav,A
2,lesred1,[],11:00,692819.wav,A
3,lesred1,[],11:00,692818.wav,A
4,lesred1,[],11:00,692817.wav,A
...,...,...,...,...,...
9770,sposan,[],?:?,51888.wav,D
9771,sposan,[],8:00,5862.wav,D
9772,sposan,[],04:45,559833.wav,E
9773,sposan,[],18:50,254627.wav,E


In [61]:
valid_audio_list = []

In [62]:
valid_audio_list = get_immediate_files(dataset_dir)

In [63]:
labels = (data['primary_label'] ).unique()
print(len(labels))

20


In [64]:
sec_labels = [re.sub('[\[\'\]]', '', x) for x in data['secondary_label']]
sec_labels = [(re.sub(r"\s+", "", x)).split(',') for x in sec_labels]

In [66]:
target_dict = {}
mid_str = 'a'
mid_values = []

In [67]:
for ind, label in enumerate(labels):
    target_dict[label] = str(ind)
    mid_values.append(mid_str + str(ind).zfill(3))

In [68]:
ind_values = list(range(len(labels)))

In [69]:
custom_dict = {'index': ind_values, 'mid':mid_values, 'display_name':labels }
df = pd.DataFrame(data=custom_dict)

In [70]:
df.to_csv(os.path.join(meta_dir, 'custom_labels.csv'))

In [71]:
#CREATE CUSTOM META BELOW

In [72]:
targets = []
maxLen = 10
durations = []

In [ ]:
for index, row in data.iterrows():
    curr_targets = [target_dict[row['primary_label']]]
    if row['secondary_label'] != '[]':
        temp_secondary = (re.sub('[\[\'\]]', '', row['secondary_label']))
        temp_secondary = (re.sub(r"\s+", "", temp_secondary)).split(',')
        #print(temp_secondary)

        secondary_targets = [target_dict[x] for x in temp_secondary]
        #curr_targets += secondary_targets
    
    #curr_targets = [str(x) for x in curr_targets]
    targets.append("-".join(curr_targets))
    file_path = dataset_dir + row['filename']
    y, sr = librosa.load(path = file_path , sr = 16000)
    durations.append(librosa.get_duration(y=y, sr=sr))

    if index % 10000 == 0:
        print(index)

In [81]:
num_fold =5
sample_per_fold = len(data)//num_fold
remainder = len(data) % num_fold

In [82]:
folds = []
for i in range(num_fold):
    for j in range(sample_per_fold):
        folds.append(i+1)

In [83]:
for j in range(remainder):
    folds.append(j+1)

In [84]:
random.shuffle(folds)

In [86]:
data['fold'] = folds
data['target'] = targets
data['durations'] = durations
#data["call_detection"] = call_binaries
data = data[["filename","fold","target","primary_label","durations"]]

In [90]:
data.to_csv(meta_dir + '/custom_data/custom_meta.csv')